## Setup

In [8]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal
from transformer_lens.utils import test_prompt
import pickle
from ipywidgets import interact, IntSlider, SelectionSlider
from transformer_lens.utils import test_prompt

import plotly.graph_objects as go

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import AutoEncoderConfig, eval_direction_tokens_global, get_encode_activations_hook, get_activations, get_acts, load_encoder, eval_ablation_token_rank, get_direction_ablation_hook, get_top_activating_examples_for_direction, evaluate_direction_ablation_single_prompt
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line, multiple_line
%reload_ext autoreload
%autoreload 2

In [9]:
model_name = "tiny-stories-2L-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)
model.set_use_attn_result(True)

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [10]:
# L0 '2_silvery_smoke',
# l1 '2_soft_monkey',  
# L3 '2_driven_planet'
run_names = ["18_morning_sun", "98_hardy_firefly"] #"142_efficient_lion"]# "98_hardy_firefly"] #"8_deep_brook"]
#run_names = ['1_skilled_universe', '47_winter_sun']
encoders = []
for run_name in run_names:
    encoder, cfg = load_encoder(run_name, model_name, model)
    cfg.run_name = run_name
    print(cfg.run_name, cfg.layer, cfg.l1_coeff)
    encoders.append((encoder, cfg))

18_morning_sun 0 0.0001
98_hardy_firefly 1 [0.0001, 0.00015]


In [11]:
prompts = load_tinystories_validation_prompts()

(INFO) 02:03:34: Loaded 21990 TinyStories validation prompts


In [12]:
max_activation_data = {}
for encoder, cfg in encoders:
    run_name = cfg.run_name
    max_activations, max_activation_token_indices = get_activations(encoder, cfg, run_name, prompts, model)
    max_activation_data[run_name] = {
        "max_activations": max_activations.cpu(),
        "max_activation_token_indices": max_activation_token_indices.cpu()
    }

In [13]:
def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [14]:
first_encoder, first_encoder_cfg = encoders[0]
second_encoder, second_encoder_cfg = encoders[1]

first_encoder_max_activations = max_activation_data[first_encoder_cfg.run_name]["max_activations"]
first_encoder_max_activation_token_indices = max_activation_data[first_encoder_cfg.run_name]["max_activation_token_indices"]
second_encoder_max_activations = max_activation_data[second_encoder_cfg.run_name]["max_activations"]
second_encoder_max_activation_token_indices = max_activation_data[second_encoder_cfg.run_name]["max_activation_token_indices"]

In [62]:
def get_common_tinystories_tokens(prompts, model: HookedTransformer, min_occurrences=100):
    occurrences = torch.zeros(model.cfg.d_vocab, dtype=torch.int32).cuda()
    for prompt in prompts: 
        tokens = model.to_tokens(prompt).flatten()
        occurrences = occurrences.index_add(0, tokens, torch.ones_like(tokens, dtype=torch.int32))
    common_tokens = torch.argwhere(occurrences > min_occurrences).flatten()
    rare_tokens = torch.argwhere(occurrences <= min_occurrences).flatten()
    return occurrences, common_tokens, rare_tokens

occurrences, common_tokens, rare_tokens = get_common_tinystories_tokens(prompts, model)
print(len(common_tokens), len(rare_tokens))

2552 47705


## Pairwise cosine circuit discovery

In [ ]:
W_out = model.W_out[first_encoder_cfg.layer]
W_in = model.W_in[second_encoder_cfg.layer]

cosine_sims = torch.nn.functional.normalize(first_encoder.W_dec @ W_out, dim=-1) @ torch.nn.functional.normalize(W_in @ second_encoder.W_enc, dim=0)
cosine_sims = torch.tril(cosine_sims)

def i_to_row_col(i: int, n_cols: int = first_encoder.d_hidden):
    row = i // n_cols
    col = i % n_cols
    return row, col

all_sims = cosine_sims.flatten().cpu()
top_cosine_similarities, top_cosine_sim_indices = torch.topk(all_sims, 10)

In [ ]:
data = []
for top_cosine_index in tqdm(top_cosine_sim_indices):
    first_encoder_dir, second_encoder_dir = i_to_row_col(top_cosine_index)
    top_prompts, top_prompt_token_indices = get_top_activating_examples_for_direction(prompts, first_encoder_dir, first_encoder_max_activations, first_encoder_max_activation_token_indices, k=100)
    
    original_losses = []
    first_encoder_losses = []
    second_encoder_losses = []
    acts = []
    ablated_acts = []
    for prompt, pos in zip(top_prompts, top_prompt_token_indices.tolist()):
        # Direction losses
        original_loss, first_encoder_ablated_loss = evaluate_direction_ablation_single_prompt(prompt, first_encoder, model, first_encoder_dir, first_encoder_cfg, pos=pos)
        _, second_encoder_ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, second_encoder_dir, second_encoder_cfg, pos=pos)
        original_losses.append(original_loss)
        first_encoder_losses.append(first_encoder_ablated_loss)
        second_encoder_losses.append(second_encoder_ablated_loss)

        # Second encoder direction activation with and without ablation
        act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[pos, second_encoder_dir].item()
        encoder_hook_point = f"blocks.{first_encoder_cfg.layer}.{first_encoder_cfg.act_name}"
        with model.hooks(fwd_hooks=[(encoder_hook_point, get_direction_ablation_hook(first_encoder, first_encoder_dir, pos))]):
            ablated_act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[pos, second_encoder_dir].item()
        acts.append(act)
        ablated_acts.append(ablated_act)

    data.append([first_encoder_dir.item(), second_encoder_dir.item(), np.mean(original_losses), np.mean(first_encoder_losses), np.mean(second_encoder_losses), np.mean(acts), np.mean(ablated_acts)])
df = pd.DataFrame(data, columns=["Encoder 1 direction", "Encoder 2 direction", "Original loss", "Encoder 1 direction ablation loss", "Encoder 2 direction ablation loss", "Second encoder activation", "Second encoder activation after ablation"])
df["Cosine similarity"] = top_cosine_similarities.tolist()

## Prompt co occurrence analysis

In [ ]:
# Pick interesting looking prompt
# Save activations of all directions for that prompt
# Save last layer active directions for each earlier direction ablated individually
# Compute AND measure for all active directions in last layer based on previous layers

In [ ]:
prompt = "This moral story teaches children that"
second_encoder_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-1]
second_encoder_top_acts, second_encoder_top_dirs = torch.topk(second_encoder_acts, 10)
second_encoder_direction = second_encoder_top_dirs[0].item()
first_encoder_acts = get_acts(prompt, model, first_encoder, first_encoder_cfg)[-1]
active_first_encoder_directions = torch.argwhere(first_encoder_acts > 1).flatten().tolist()
original_second_encoder_act = second_encoder_top_acts[0].item()
#px.histogram(acts.cpu().numpy(), width=700)

In [ ]:
data = []
hook_point = first_encoder_cfg.encoder_hook_point
for first_encoder_direction in active_first_encoder_directions:
    ablation_hook = get_direction_ablation_hook(first_encoder, first_encoder_direction, -1)
    with model.hooks([(hook_point, ablation_hook)]):
        ablated_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-1, second_encoder_direction].item()
    data.append([first_encoder_direction, ablated_acts])
and_df = pd.DataFrame(data, columns=["First encoder direction", "Second encoder activation after ablation"])
and_df["Activation difference"] = and_df["Second encoder activation after ablation"] - original_second_encoder_act
and_df

In [ ]:
data = []
hook_point = first_encoder_cfg.encoder_hook_point
for prompt_index, prompt in enumerate(prompts[:2]):
    second_encoder_acts_all_pos = get_acts(prompt, model, second_encoder, second_encoder_cfg)
    first_encoder_acts_all_pos = get_acts(prompt, model, first_encoder, first_encoder_cfg)
    num_tokens = second_encoder_acts_all_pos.shape[0]
    for position in range(10, num_tokens):
        first_encoder_acts = first_encoder_acts_all_pos[position]
        second_encoder_acts = second_encoder_acts_all_pos[position]

        second_encoder_top_acts, second_encoder_top_dirs = torch.topk(second_encoder_acts, 10)
        second_encoder_direction = second_encoder_top_dirs[0].item()
        active_first_encoder_directions = torch.argwhere(first_encoder_acts > 1).flatten().tolist()
        original_second_encoder_act = second_encoder_top_acts[0].item()

        for first_encoder_direction in active_first_encoder_directions:
            ablation_hook = get_direction_ablation_hook(first_encoder, first_encoder_direction, position)
            with model.hooks([(hook_point, ablation_hook)]):
                ablated_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[position, second_encoder_direction].item()
            data.append([prompt_index, position, first_encoder_direction, second_encoder_direction, ablated_acts, original_second_encoder_act])
and_df = pd.DataFrame(data, columns=["Prompt", "Position", "First encoder direction", "Second encoder direction", "Second encoder activation after ablation", "Second encoder activation"])
and_df["Activation difference"] = and_df["Second encoder activation after ablation"] - and_df["Second encoder activation"]
and_df = and_df.sort_values("Activation difference", ascending=False)

In [ ]:
and_df.head(10)

In [ ]:
print_top_examples(prompts, second_encoder_max_activations, second_encoder_direction, second_encoder, second_encoder_cfg, n=1)

## Get top token occurrences per direction

In [170]:
def get_direction_token_df(max_activations, prompts, model, encoder, encoder_cfg, percentage_threshold=0.5, save_path="/workspace/data/top_token_occurrences"):
    os.makedirs(save_path, exist_ok=True)
    file_name = f"{save_path}/{encoder_cfg.run_name}_direction_token_occurrences.csv"
    if os.path.exists(file_name):
        direction_df = pd.read_csv(file_name)
    else:

        token_wise_activations = eval_direction_tokens_global(max_activations, prompts, model, encoder, encoder_cfg, percentage_threshold=0.5)
        total_occurrences = token_wise_activations.sum(1)
        max_occurrences = token_wise_activations.max(1)[0]
        max_occurring_token = token_wise_activations.argmax(1)
        str_tokens = model.to_str_tokens(torch.LongTensor(list(range(model.cfg.d_vocab))))

        direction_data = []
        for direction in tqdm(range(encoder.d_hidden)):
            total_occurrence = total_occurrences[direction].item()
            top_occurrence = max_occurrences[direction].item()
            top_token = model.to_single_str_token(max_occurring_token[direction].item())
            direction_data.append([direction, total_occurrence, top_token, top_occurrence])

        direction_df = pd.DataFrame(direction_data, columns=["Direction", "Total occurrences", "Top token", "Top token occurrences"])
        direction_df["Top token percent"] = direction_df["Top token occurrences"] / direction_df["Total occurrences"]
        direction_df = direction_df.dropna()

    print(len(direction_df))
    return direction_df

direction_df = get_direction_token_df(first_encoder_max_activations, prompts, model, first_encoder, first_encoder_cfg, percentage_threshold=0.5)

  1%|          | 215/21990 [00:06<10:14, 35.44it/s]


KeyboardInterrupt: 

In [ ]:
fig = px.histogram(direction_df, x="Top token percent", width=700, title="Per direction percentage of activations on top token")
fig.update_layout({
    "xaxis_title": "Top token activation percentage",
})
fig.show()

In [ ]:
#good_directions = direction_df[(direction_df["Top token percent"] > 0.2) & (direction_df["Top token percent"] < 0.7)]["Direction"].tolist()
good_directions = direction_df[(direction_df["Top token percent"] < 0.2)]["Direction"].tolist()

print(len(good_directions))

2629


In [ ]:
def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().unique().tolist()
    max_direction_act = activations[:, direction].max().item()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        if max(direction_act) > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

def print_direction_example(direction, n=10):
    print_top_examples(prompts, first_encoder_max_activations, direction, first_encoder, first_encoder_cfg, n)

# Max activations
interact(print_direction_example, 
         direction=SelectionSlider(options=good_directions, value=good_directions[0], description='Direction'),
         #direction=IntSlider(min=0, max=l0_encoder.d_hidden-1, step=1, value=0),
         n=IntSlider(min=1, max=20, step=1, value=5))

interactive(children=(SelectionSlider(description='Direction', options=(5, 11, 16, 20, 24, 29, 30, 33, 38, 46,…

<function __main__.print_direction_example(direction, n=10)>

## Quotation mark

In [15]:
# Capabilities
# Start quotation after "said" or ":" or other obvious tokens (check tokenization)
# End quotation after "." if started

# Model can definitely predict 'said, "' trigram
#test_prompt(test_prompts[1][:-2], " \"", model)

In [16]:
# '."' '?"' and '!"' are single tokens
# Save prompts where next token is '."'

answer_token = model.to_single_token(".\"")
test_prompts = []
for prompt in tqdm(prompts):
    if "said, \"" in prompt:
        start_index = prompt.index("said, \"") + 7
        end_index = prompt.find(".\"", start_index)
        if end_index != -1:
            subprompt = prompt[:end_index+2]
            tokens = model.to_tokens(subprompt)
            last_token = model.to_single_str_token(tokens[0, -1].item())
            if (subprompt[-2:] == ".\"") and (last_token == ".\""):
                test_prompts.append(subprompt)
print(len(test_prompts))


  0%|          | 0/21990 [00:00<?, ?it/s]

4554


In [17]:
print(model.to_str_tokens(model.to_tokens(test_prompts[0])))

['<|endoftext|>', 'Spot', '.', ' Spot', ' saw', ' the', ' shiny', ' car', ' and', ' said', ',', ' "', 'Wow', ',', ' Kitty', ',', ' your', ' car', ' is', ' so', ' bright', ' and', ' clean', '!"', ' Kitty', ' smiled', ' and', ' replied', ',', ' "', 'Thank', ' you', ',', ' Spot', '.', ' I', ' polish', ' it', ' every', ' day', '."']


In [18]:
#test_prompt(test_prompts[0][:-2], '."', model, prepend_space_to_answer=False)

In [19]:
def DLA(prompts: list[str], model: HookedTransformer, pos=-1) -> tuple[Float[Tensor, "component"], list[str]]:
    logit_attributions = []
    for prompt in tqdm(prompts):
        tokens = model.to_tokens(prompt)
        answers = tokens[:, 1:]
        tokens = tokens[:, :-1]
        answer_residual_directions = model.tokens_to_residual_directions(answers)[0, pos]  # [batch pos d_model]
        _, cache = model.run_with_cache(tokens)
        accumulated_residual, labels = cache.get_full_resid_decomposition(layer=-1, pos_slice=pos, return_labels=True, expand_neurons=False)
        scaled_residual_stack = cache.apply_ln_to_stack(accumulated_residual, layer = -1, pos_slice=pos).squeeze(1)
        logit_attribution = einops.einsum(scaled_residual_stack, answer_residual_directions, "component d_model, d_model -> component") # / answers.shape[0]
        logit_attributions.append(logit_attribution)
    
    logit_attributions = torch.stack(logit_attributions).mean(0)
    return logit_attributions, labels

In [20]:
# DLA of '."', find relevant MLPs if they exist, check encoders of those MLPs for relevant directions
# DLA looks pretty different for different prompts
# Relevant boosting heads:
# Relevant boosting MLP: 
# Relevant deboosting heads: 
# Relevant deboosting MLP:
dlas, labels = DLA(test_prompts, model, pos=-1)
print(dlas.shape)

  0%|          | 0/4554 [00:00<?, ?it/s]

torch.Size([37])


In [21]:
line(dlas.cpu().numpy(), xticks=labels, width=1200, title="Closing quotation DLA", show_legend=False)

In [22]:
second_encoder_cfg

AutoEncoderConfig(layer=1, act_name='mlp.hook_post', expansion_factor=4, l1_coeff=[0.0001, 0.00015], d_in=4096, run_name='98_hardy_firefly', reg='combined_hoyer_sqrt')

In [23]:
all_acts = []
for prompt in test_prompts[:1000]:
    acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-2]
    all_acts.append(acts)

all_acts = torch.stack(all_acts)

# Max per direction
max_val, _ = second_encoder_max_activations.max(0)
threshold_per_direction = (max_val * 0.17).cuda()

# Mean activation on all prompts is misleading, prompts could be important on subset of test prompts
num_active_acts = (all_acts > threshold_per_direction).sum(0) + 1e-9
all_acts_tmp = all_acts.clone()
all_acts_tmp[all_acts_tmp <= threshold_per_direction] = 0
# Direction wise mean activation on active prompts
mean_active_acts = all_acts_tmp.sum(0) / num_active_acts
# Filter directions that are active on less than x% of quotation prompts
mean_active_acts[num_active_acts < 0.05*all_acts.shape[0]] = 0
n_non_zero_directions = (mean_active_acts > 0).sum().item()
top_acts, top_dirs = torch.topk(mean_active_acts, min(100, n_non_zero_directions))
print(len(top_acts), top_acts, top_dirs)

16 tensor([3.4235, 3.3499, 3.3032, 2.7875, 2.6495, 2.5719, 2.4340, 2.4006, 2.0264,
        1.9179, 1.8494, 1.8454, 1.7016, 1.6670, 1.6152, 1.5300],
       device='cuda:0') tensor([13967, 10593,  8093, 15796, 14511,  3136,   745,  5765,  8675,   528,
         6366,   219, 11854,  2273, 12392,  9924], device='cuda:0')


In [24]:
fig = px.histogram(all_acts.mean(0).cpu().numpy(), width=700)
fig.update_layout({
    "showlegend": False,
    "xaxis_title": "Encoder activations",
    "title": "Encoder activations before closing quotation prompts"
})
fig.show()

In [25]:
# Filter test prompts by which prompts the direction activates on
activating_test_prompts_all_dir = torch.zeros((len(test_prompts), second_encoder.d_hidden), dtype=torch.bool)
for i, prompt in tqdm(enumerate(test_prompts), total=len(test_prompts)):
    tokens = model.to_tokens(prompt)
    act_token_index = tokens.shape[1] - 2
    act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[act_token_index]
    act_active = act > 0.1
    activating_test_prompts_all_dir[i] = act_active

  0%|          | 0/4554 [00:00<?, ?it/s]

In [26]:
# Run ablation for activating directions
data = []
for direction in tqdm(top_dirs):
    loss_increases = []
    active_test_prompt_indices = torch.argwhere(activating_test_prompts_all_dir[:, direction]).flatten().tolist()
    active_test_prompts = [test_prompts[i] for i in active_test_prompt_indices]
    num_prompts = min(len(active_test_prompts), 200)
    for prompt in active_test_prompts[:num_prompts]:
        tokens = model.to_tokens(prompt)
        pos = tokens.shape[1]-2
        original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, direction.item(), second_encoder_cfg, pos=pos)
        loss_increase = ablated_loss - original_loss
        loss_increases.append(loss_increase)
    loss_increase = np.mean(loss_increases)
    mean_activation =  mean_active_acts[direction].item()
    percentage_activation = num_active_acts[direction].item() / all_acts.shape[0]
    data.append([direction.item(), loss_increase, mean_activation, percentage_activation])
df = pd.DataFrame(data, columns=["Direction", "Loss increase", "Mean activation", "Percentage activation"])
df.sort_values("Loss increase", ascending=False).head(10)

  0%|          | 0/16 [00:00<?, ?it/s]

,Direction,Loss increase,Mean activation,Percentage activation
3,15796,0.559039,2.787491,0.855
2,8093,0.307295,3.303170,0.473
1,10593,0.270368,3.349861,0.094
0,13967,0.082636,3.423550,0.050
6,745,0.065768,2.434007,0.131
10,6366,0.044746,1.849365,0.058
5,3136,0.035581,2.571891,0.093
7,5765,0.000091,2.400604,0.059
9,528,-0.000877,1.917923,0.398
12,11854,-0.001141,1.701566,0.067


In [91]:
df = df.sort_values("Loss increase", ascending=False)
top_directions = df["Direction"].tolist()
directions = top_directions[:3]
print(top_directions)
df.head(10)

[15796, 8093, 10593, 13967, 745, 6366, 3136, 5765, 528, 11854, 14511, 219, 8675, 2273, 9924, 12392]


,Direction,Loss increase,Mean activation,Percentage activation
3,15796,0.559039,2.787491,0.855
2,8093,0.307295,3.303170,0.473
1,10593,0.270368,3.349861,0.094
0,13967,0.082636,3.423550,0.050
6,745,0.065768,2.434007,0.131
10,6366,0.044746,1.849365,0.058
5,3136,0.035581,2.571891,0.093
7,5765,0.000091,2.400604,0.059
9,528,-0.000877,1.917923,0.398
12,11854,-0.001141,1.701566,0.067


In [28]:
# Compute MLP remainder
# DLA of MLP compared to DLA of reconstructed MLP
# Mean activations over a bunch of test prompts
# Mean reconstructions over a bunch of test prompts
# Apply LN final layer to both
# Compute DLA of both

dlas = []
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"
for prompt in test_prompts[:200]:
    _, cache = model.run_with_cache(prompt)
    mlp_acts = cache[hook_name][0, -2]
    
    _, x_reconstruct, _, _, _ = second_encoder(mlp_acts)
    
    mlp_res = mlp_acts @ model.W_out[second_encoder_cfg.layer]
    reconstruct_res = x_reconstruct @ model.W_out[second_encoder_cfg.layer]
    remainder = (mlp_res - reconstruct_res) 

    stacked_residual = torch.stack([mlp_res, reconstruct_res, remainder])
    stacked_residual = cache.apply_ln_to_stack(stacked_residual, pos_slice=-2)

    dla = stacked_residual @ model.W_U[:, answer_token]

    dlas.append(dla)

dla = torch.stack(dlas).mean(0)
print(dla.shape)


torch.Size([3])


In [29]:
line(dla.tolist(), xticks=["MLP", "Reconstruct", "Remainder"], show_legend=False, title="Last MLP layer DLA for closing quotation prompts")

In [30]:
# Check neuron basis for directions
# Cosine sims of direction encoder weights
# Check if directions activate together or on separate examples
# Compare set of top 100 contributing neurons for each direction for overlap
# DFA for directions

In [97]:
def direction_dla(direction, max_activations, max_activation_token_indices, encoder, encoder_cfg, n=100, mean_mlp_decomp= None):
    num_non_zero_activations = max_activations[:, direction].nonzero().shape[0]
    top_prompts, top_prompt_token_indices = get_top_activating_examples_for_direction(prompts, direction, max_activations, max_activation_token_indices, k=num_non_zero_activations, mode="top")
    
    direction_weight = encoder.W_enc[:, direction]
    dlas = []
    for i in range(n):
        prompt = top_prompts[i]
        pos = top_prompt_token_indices[i]
        _, cache = model.run_with_cache(prompt)

        decomposition, labels = cache.get_full_resid_decomposition(encoder_cfg.layer, mlp_input=True, apply_ln=True, return_labels=True, expand_neurons=False, pos_slice=pos)
        decomposition = decomposition.squeeze(1)

        # Account for GELU in DLA by setting neuron contributions to 0 if they are not activated
        mlp_wise_decomposition = einops.einsum(decomposition, model.W_in[encoder_cfg.layer], "component d_res, d_res d_mlp -> component d_mlp")
        mlp_activations = cache[f"blocks.{encoder_cfg.layer}.mlp.hook_post"][0, pos, :]
        zeroed_neurons = torch.argwhere(mlp_activations <= 0).flatten()
        if mean_mlp_decomp is not None:
            mlp_wise_decomposition[:, zeroed_neurons] = mean_mlp_decomp[:, zeroed_neurons]
        else:
            mlp_wise_decomposition[:, zeroed_neurons] = 0

        dla = einops.einsum(mlp_wise_decomposition, direction_weight, "component d_mlp, d_mlp -> component")
        dlas.append(dla)
    dla = torch.stack(dlas).mean(0).tolist()
    return dla, labels

def get_mean_component_wise_mlp(prompts, encoder_cfg):
    mlp_wise_decompositions = []
    for prompt in prompts:
        _, cache = model.run_with_cache(prompt)

        decomposition = cache.get_full_resid_decomposition(encoder_cfg.layer, mlp_input=True, apply_ln=True, return_labels=False, expand_neurons=False, pos_slice=None)
        decomposition = decomposition.squeeze(1) # Batch
        # Account for GELU in DLA by setting neuron contributions to 0 if they are not activated
        mlp_wise_decomposition = einops.einsum(decomposition, model.W_in[encoder_cfg.layer], "component pos d_res, d_res d_mlp -> component pos d_mlp")
        mlp_wise_decomposition = mlp_wise_decomposition.mean(1)
        mlp_wise_decompositions.append(mlp_wise_decomposition)
    mlp_wise_decompositions = torch.stack(mlp_wise_decompositions).mean(0)
    return mlp_wise_decompositions

dlas = []
mean_mlp_decomp = get_mean_component_wise_mlp(prompts[:100], second_encoder_cfg)
for direction in directions:
    dla, labels = direction_dla(direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, second_encoder, second_encoder_cfg, mean_mlp_decomp=mean_mlp_decomp)
    dlas.append(dla)
multiple_line(dlas, directions, xticks=labels, title="DFA for top directions (mean ablation of inactive MLP neurons)", width=1000)

In [96]:
dlas = []
for direction in directions:
    dla, labels = direction_dla(direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, second_encoder, second_encoder_cfg, mean_mlp_decomp=None)
    dlas.append(dla)
multiple_line(dlas, directions, xticks=labels, title="DFA for top directions (zero ablation of inactive MLP neurons)", width=1000)

In [33]:
prompt = active_test_prompts[0]
_, cache = model.run_with_cache(prompt)
pattern = cache["pattern", 1][:, 6]
print(pattern.shape)

torch.Size([1, 99, 99])


In [34]:
# import circuitsvis as cv
# display(cv.attention.attention_patterns(
#         attention = pattern.cpu(),
#         tokens = model.to_str_tokens(prompt),
#         attention_head_names = ["L1H6"],
#     ))

In [109]:
# Correlation between activating test prompts
direction_activations = activating_test_prompts_all_dir[:, directions].to(torch.int32)
correlation_matrix = torch.corrcoef(direction_activations.T)
print(correlation_matrix.triu(1))

tensor([[ 0.0000, -0.0458, -0.0667],
        [ 0.0000,  0.0000, -0.0421],
        [ 0.0000,  0.0000,  0.0000]])


## Ablate set of important L1 directions

In [35]:
#directions = [3373, 15330, 2594, 8842, 7447]
directions = top_directions[:3]
print(len(directions), directions)

3 [15796, 8093, 10593]


In [36]:
# On positions which close quotation
loss_increases = []
for prompt in test_prompts[:200]:
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, directions, second_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.8348632690031081 0.634459187936737


In [37]:
answer_token = ".\""
n = 200
data = []
for test_prompt_index, prompt in tqdm(enumerate(test_prompts[:n]), total=n):
    answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank = eval_ablation_token_rank(prompt, second_encoder, model, directions, second_encoder_cfg, answer_token, pos=-2)
    data.append([test_prompt_index, answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank])
ablation_df = pd.DataFrame(data, columns=["Prompt index", "Answer logprob", "Ablated answer logprob", "Answer logit", "Ablated answer logit", "Answer rank", "Ablated answer rank"])
# Calculate differences between original and ablated measures
ablation_df['Logprob Difference'] = ablation_df['Answer logprob'] - ablation_df['Ablated answer logprob']
ablation_df['Logit Difference'] = ablation_df['Answer logit'] - ablation_df['Ablated answer logit']
ablation_df['Rank Difference'] = ablation_df['Answer rank'] - ablation_df['Ablated answer rank']

  0%|          | 0/200 [00:00<?, ?it/s]

In [38]:
def plot_ablation_losses(ablation_df):
    # Calculate means, standard deviations, and standard errors for differences
    means_diff = ablation_df[['Logprob Difference', 'Logit Difference', 'Rank Difference']].mean()
    stds_diff = ablation_df[['Logprob Difference', 'Logit Difference', 'Rank Difference']].std()

    # Standard Errors
    stderrs_diff = stds_diff / np.sqrt(n)

    # 95% Confidence Intervals for Differences
    confidence_interval_diff = 1.96 * stderrs_diff

    # Lower and upper bounds for differences
    lower_bounds_diff = means_diff - confidence_interval_diff
    upper_bounds_diff = means_diff + confidence_interval_diff

    # Create a bar plot for differences
    fig = go.Figure()
    metrics = ['Logprob Difference', 'Logit Difference', 'Rank Difference']

    for metric in metrics:
        fig.add_trace(go.Bar(
            x=[metric],
            y=[means_diff[metric]],
            error_y=dict(
                type='data',  # Represents actual data points
                array=[upper_bounds_diff[metric] - means_diff[metric]],
                arrayminus=[means_diff[metric] - lower_bounds_diff[metric]]
            ),
            name=metric
        ))

    fig.update_layout(
        title="Original - Ablated Measures",
        xaxis_title="Metric",
        yaxis_title="Difference",
        barmode='group',
        width=700
    )

    fig.show()


In [39]:
plot_ablation_losses(ablation_df)

In [40]:
# Baseline check: mean ablate MLP1
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"
mean_activation = []
for prompt in prompts[:10]:
    _, cache = model.run_with_cache(prompt)
    activations = cache[hook_name][0].mean(0)
    mean_activation.append(activations)
mean_activation = torch.stack(mean_activation).mean(0)

In [41]:
def mean_ablation_hook(value, hook):
    value[0, -2] = mean_activation 
    return value

In [42]:
loss_increases = []
answer_token_index = model.to_single_token(answer_token)
pos = -2
data = []
for prompt in test_prompts[:100]:
    logits = model(prompt, return_type="logits")[0, pos]
    answer_rank = (logits > logits[answer_token_index]).sum().item()
    answer_logprob = logits.log_softmax(dim=-1)[answer_token_index].item()
    answer_logit = logits[answer_token_index].item()
    with model.hooks(fwd_hooks=[(hook_name, mean_ablation_hook)]):
        ablated_logits = model(prompt, return_type="logits")[0, pos]
        ablated_answer_rank = (ablated_logits > ablated_logits[answer_token_index]).sum().item()
        ablated_answer_logprob = ablated_logits.log_softmax(dim=-1)[answer_token_index].item()
        ablated_answer_logit = ablated_logits[answer_token_index].item()
    data.append([test_prompt_index, answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank])
ablation_df = pd.DataFrame(data, columns=["Prompt index", "Answer logprob", "Ablated answer logprob", "Answer logit", "Ablated answer logit", "Answer rank", "Ablated answer rank"])
# Calculate differences between original and ablated measures
ablation_df['Logprob Difference'] = ablation_df['Answer logprob'] - ablation_df['Ablated answer logprob']
ablation_df['Logit Difference'] = ablation_df['Answer logit'] - ablation_df['Ablated answer logit']
ablation_df['Rank Difference'] = ablation_df['Answer rank'] - ablation_df['Ablated answer rank']
plot_ablation_losses(ablation_df)

## Loss recovered analysis

In [43]:
# Baseline check: mean ablate MLP1
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"

mean_activation = []
for prompt in prompts[:20]:
    _, cache = model.run_with_cache(prompt)
    activations = cache[hook_name][0].mean(0)
    mean_activation.append(activations)
mean_activation = torch.stack(mean_activation).mean(0)

def mean_ablation_hook(value, hook):
    value[0, -2] = mean_activation 
    return value

def zero_ablation_hook(value, hook):
    value[0, -2] = 0
    return value

def encode_activations_hook(value, hook):
    _, x_reconstruct, acts, _, _ = second_encoder(value[:, -2])
    value[:, -2] = x_reconstruct
    return value

ablate_top_directions_hook = [(hook_name, get_direction_ablation_hook(second_encoder, directions, -2))]
encode_mlp_hook = [(hook_name, encode_activations_hook)]

In [44]:
# Zero ablate MLP
# ((zero_abl_loss - recons_loss)/(zero_abl_loss - loss))
# Mean ablate MLP
# Enable only top 2 directions
# Enable full autoencoder

losses = []
mean_abl_losses = []
zero_abl_losses = []
top_abl_losses = []
recons_losses = []

for prompt in tqdm(test_prompts):
    loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=[(hook_name, mean_ablation_hook)]):
        mean_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=[(hook_name, zero_ablation_hook)]):
        zero_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=ablate_top_directions_hook):
        top_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=encode_mlp_hook):
        recons_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    losses.append(loss)
    mean_abl_losses.append(mean_abl_loss)
    zero_abl_losses.append(zero_abl_loss)
    top_abl_losses.append(top_abl_loss)
    recons_losses.append(recons_loss)


  0%|          | 0/4554 [00:00<?, ?it/s]

In [45]:
import plotly.graph_objects as go
import numpy as np

# Calculate means and standard deviations
means = [
    np.mean(losses),
    np.mean(mean_abl_losses),
    np.mean(zero_abl_losses),
    np.mean(top_abl_losses),
    np.mean(recons_losses)
]

def standard_error(data):
    return np.std(data) / np.sqrt(len(data))

std_errors = [
    standard_error(losses),
    standard_error(mean_abl_losses),
    standard_error(zero_abl_losses),
    standard_error(top_abl_losses),
    standard_error(recons_losses)
]

ci_95 = [se * 1.96 for se in std_errors]

labels = ['Original loss', 'MLP mean ablation loss', 'MLP zero ablation loss', f'Top {len(directions)} SAE directions ablation loss', 'SAE reconstruction loss']
fig = go.Figure(data=[
    go.Bar(
        x=labels,
        y=means,
        error_y=dict(type='data', array=ci_95)
    )
])

fig.update_layout(
    title="Ablation loss comparison for closing quotation prompts '.\"'",
    xaxis_title="Ablation",
    yaxis_title="Loss",
    showlegend=False,
    width=600
)

fig.show()


In [46]:
loss_recovered = (np.mean(zero_abl_losses) - np.mean(recons_losses)) / (np.mean(zero_abl_losses) - np.mean(losses))
print(loss_recovered)

0.8266587223460056


## Directions

In [124]:
direction = directions[0]
print(direction)

15796


In [125]:
from ipywidgets import interactive, IntSlider, SelectionSlider

def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

# def print_direction_example(direction, num_examples=3):
#     print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, num_examples)
# widget = interactive(print_direction_example, 
#              direction=SelectionSlider(options=directions, value=directions[0], description='Direction'),
#              num_examples=IntSlider(min=1, max=20, step=1, value=5))
# display(widget)

print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, 2)

In [130]:
# Filter test prompts by which prompts the direction activates on
activating_test_prompt_indices = activating_test_prompts_all_dir[:, direction].nonzero().flatten().tolist()
activating_test_prompts = [test_prompts[i] for i in activating_test_prompt_indices]
print(len(activating_test_prompts), "out of", activating_test_prompts_all_dir.shape[0])

4442 out of 4554


In [127]:
answer_token = model.to_single_token(".\"")
quotation_tokens = ["\"", ".", "!", "?", ".\"", "!\"", "?\""]
for quotation_token in quotation_tokens:
    assert model.to_single_token(quotation_token) in common_tokens.tolist()

print(answer_token, model.to_single_str_token(answer_token))

526 ."


In [128]:
# Logprob boosts (prompt based)
def get_direction_logit_and_logprob_boost(
    prompts: list[str],
    encoder: AutoEncoder,
    encoder_neuron,
    model: HookedTransformer,
    all_ignore: Int[Tensor, "tokens"],
    cfg: AutoEncoderConfig,
    pos: -2
):
    zero_direction_hook = [(f"blocks.{cfg.layer}.{cfg.act_name}", get_direction_ablation_hook(
        encoder, encoder_neuron, pos
    ))]

    logprobs_active = []
    logprobs_inactive = []
    for prompt in prompts:
        logits_active = model(prompt, return_type="logits")[0, pos]
        with model.hooks(zero_direction_hook):
            logits_inactive = model(prompt, return_type="logits")[0, pos]

        logprobs_active_current = logits_active.log_softmax(dim=-1)
        logprobs_inactive_current = logits_inactive.log_softmax(dim=-1)
        logprobs_active.append(logprobs_active_current)
        logprobs_inactive.append(logprobs_inactive_current)
    logprobs_active = torch.stack(logprobs_active).mean(0)
    logprobs_inactive = torch.stack(logprobs_inactive).mean(0)
    print(logprobs_active[answer_token], logprobs_inactive[answer_token])

    boosts = (logprobs_active - logprobs_inactive)
    boosts[logprobs_active < -9] = 0
    boosts[all_ignore] = 0
    top_boosts, top_tokens = torch.topk(boosts, 15)
    non_zero_boosts = top_boosts != 0
    top_deboosts, top_deboosted_tokens = torch.topk(boosts, 15, largest=False)
    non_zero_deboosts = top_deboosts != 0
    boosted_tokens = (
        model.to_str_tokens(top_tokens[non_zero_boosts]),
        top_boosts[non_zero_boosts].tolist(),
    )
    deboosted_tokens = (
        model.to_str_tokens(top_deboosted_tokens[non_zero_deboosts]),
        top_deboosts[non_zero_deboosts].tolist(),
    )
    logging.info(f"Top boosted: {boosted_tokens}")
    logging.info(f"Top deboosted: {deboosted_tokens}")

get_direction_logit_and_logprob_boost(activating_test_prompts[:100], second_encoder, direction, model, rare_tokens, second_encoder_cfg, pos=-2)

(INFO) 03:18:26: Top boosted: (['!"', '."', '".'], [1.0913341045379639, 0.5869712233543396, 0.1926875114440918])
(INFO) 03:18:26: Top deboosted: ([',"', ' so', ' with', ' for', ',', ' in', ' to', ' when', '.', ' and', ' too'], [-2.0147781372070312, -1.1497187614440918, -1.1056833267211914, -1.1019926071166992, -1.0723686218261719, -1.0663819313049316, -1.0409903526306152, -0.9290275573730469, -0.8422956466674805, -0.752955436706543, -0.5734338760375977])


tensor(-0.7338, device='cuda:0') tensor(-1.3207, device='cuda:0')


In [119]:
# Logit boosts (weight based DLA)
def get_direction_boosted_tokens(direction, encoder: AutoEncoder, model: HookedTransformer, cfg: AutoEncoderConfig, rare_tokens: Tensor):
    token_boosts = encoder.W_dec[direction] @ model.W_out[cfg.layer] @ model.unembed.W_U
    token_boosts[rare_tokens] = 0
    return token_boosts

def print_token_boosts(boosts, tokens):
    str_tokens = model.to_str_tokens(tokens)
    boost_str = ""
    for token, boost in zip(str_tokens, boosts.tolist()):
        boost_str += f"('{token}': {boost:.2f}), "
    print(boost_str[:-2])

boosts = get_direction_boosted_tokens(direction, second_encoder, model, second_encoder_cfg, rare_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25)
print_token_boosts(top_boosts, top_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25, largest=False)
print_token_boosts(top_boosts, top_tokens)

('!"': 0.73), ('."': 0.57), ('!".': 0.54), ('".': 0.46), (' here': 0.29), (' today': 0.29), (' I': 0.24), (' right': 0.24), (' then': 0.24), (' too': 0.23), ('?"': 0.22), (' anyway': 0.22), (' soon': 0.22), (' now': 0.22), (' sometimes': 0.22), (' next': 0.21), (' yours': 0.20), (' or': 0.20), (' my': 0.19), (' your': 0.19), (' we': 0.19), ('in': 0.19), (' if': 0.19), (' later': 0.18), (' and': 0.18)
(' hadn': -0.27), (' couldn': -0.26), (' she': -0.25), ('
': -0.25), (' herself': -0.24), ('irled': -0.23), (' her': -0.21), (' felt': -0.21), (' refused': -0.20), (' had': -0.20), (',"': -0.20), (' popular': -0.19), (' She': -0.19), (' tasted': -0.19), (' him': -0.19), (' he': -0.19), (' himself': -0.19), (' sat': -0.18), ('agged': -0.18), (' realised': -0.18), (' sighed': -0.18), (' leaned': -0.18), ('?': -0.18), (' their': -0.17), (' them': -0.17)


## Additional ablations

In [138]:
# Does ablating the feature increase loss when the feature is active?

# On positions which close quotation
loss_increases = []
for prompt in tqdm(activating_test_prompts):
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, direction, second_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))
px.histogram(loss_increases, width=700, title=f"Loss increase for removing direction {direction} on closing quotation prompts")

  0%|          | 0/4442 [00:00<?, ?it/s]

In [135]:
test_prompt_activations = []
for prompt in tqdm(activating_test_prompts):
    act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-2, direction]
    test_prompt_activations.append(act.item())

  0%|          | 0/4442 [00:00<?, ?it/s]

In [136]:
# Histogram of activation x loss increases to check if high loss increase on a prompt just comes from the direction being very active
fig = px.scatter(x=test_prompt_activations, y=loss_increases, width=700, title="Loss increase vs activation for closing quotation prompts")
fig.update_layout({
    "xaxis_title": "Activation",
    "yaxis_title": "Loss increase"
})

In [137]:
## Look at high loss increase prompts
top_loss_increases, top_loss_increase_prompt_indices = torch.topk(torch.tensor(loss_increases), 10)

def print_examples(prompts: list[str]):
    for prompt in prompts:
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

print_examples([activating_test_prompts[i] for i in top_loss_increase_prompt_indices])

In [ ]:
# In general on max activating prompts, ablating globally
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
for prompt, index in zip(max_activating_prompts, max_activating_token_indices.tolist()):
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.006354030966758728 0.00835937654341787


In [ ]:
# In general on max activating prompts, ablating one activating position at a time
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
threshold = second_encoder_max_activations[:, direction].max() * 0.1

for prompt, index in tqdm(zip(max_activating_prompts, max_activating_token_indices.tolist()), total=len(max_activating_prompts)):
    acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[:, direction]
    # No loss for last position, exclude
    active_positions = torch.argwhere(acts[:-1] > threshold).flatten().tolist()
    for position in active_positions:
        original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg, pos=position)
        loss_increase = ablated_loss - original_loss
        loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

  0%|          | 0/100 [00:00<?, ?it/s]

0.10963073582908171 0.4840503690021192


## Check for directions with positive DLA


In [77]:
activation_scaled_direction_dla = (all_acts.mean(0).unsqueeze(1) * second_encoder.W_dec) @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
top_acts, top_dirs = torch.topk(activation_scaled_direction_dla, 5)
print(top_acts, top_dirs)

torch.Size([16384])
tensor([1.4160, 0.9038, 0.1925, 0.1394, 0.1394], device='cuda:0') tensor([15796,  8093, 10593, 16264,   745], device='cuda:0')


In [89]:
direction_dla = second_encoder.W_dec @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
active_directions = (all_acts.mean(0) > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=direction_dla.cpu()[active_directions], y=all_acts.mean(0).cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="Direction DLA vs activation",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(154)


In [88]:
active_directions = (all_acts.mean(0) > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=activation_scaled_direction_dla.cpu()[active_directions], y=all_acts.mean(0).cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="DLA and activation of active Layer 1 encoder directions",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(154)
